In [ ]:
import socketio
soc = socketio.Client()

import logging
logging.basicConfig(level=logging.DEBUG)

import ifcopenshell

d = {}

@soc.event
def connect():
    print('connected to Jupyter coding frontend')

soc.connect("http://127.0.0.1:8080", transports=['polling'])
soc.emit('feeds', 'Hello from Jupyter coding frontend!')

@soc.event
def user_opens(data):
    d['model_path'] = data['model_path']
    d['ifc'] = ifcopenshell.open(data['model_path'])
    print('user_opens', d['ifc'])

@soc.on('highlighting')
def get_highlighted(data):
    print(data)
    d['guids'] = data['guids']


# soc.wait()

In [2]:
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.api.pset
import ifcopenshell.util.element
import pandas as pd

In [ ]:
def get_psets(e): return ifcopenshell.util.element.get_psets(e, psets_only=True)
def get_qtos(e): return ifcopenshell.util.element.get_psets(e, qtos_only=True)


In [ ]:
import gui_mulproc_table as gmt
import importlib
importlib.reload(gmt)
# Read selected/highlighted element from frontend
ifc = d['ifc']
guids = d['guids']
for guid in d['guids']:
    e = ifc.by_guid(guid)
    dct = dict(sorted(get_psets(e).items()))

    dfs = {}
    for k, pset in dct.items():
        print(k, pset)
        pst = pd.DataFrame.from_dict(pset, 'index')
        print(pst)
        pst = pst.reset_index()
        #pst.set_index(pst.index)
        dfs[k] = pst

    exported_data = gmt.run_tkinter_gui(dfs)

    # agt.show_dataframes_as_tabs(dfs)
    # for k, pset in get_psets(e).items():
    #     print(k, pset)
    #     print(pd.DataFrame.from_dict(pset, orient='index'))

In [ ]:
exported_data